# 05-agent-weather

05 - Agent Weather (LangChain 1.0 版本)
使用 create_agent API 创建智能体，让 AI 能够使用工具获取实时天气信息

## 运行说明

本 Notebook 与对应的 Python 文件内容一致。
如需运行完整代码，请使用 Python 脚本：
```bash
python agent_weather.py
```

In [ ]:
# 导入必要的库
import os
import sys
from pathlib import Path

# 添加项目根目录到路径
sys.path.insert(0, str(Path.cwd()))

from dotenv import load_dotenv
load_dotenv(override=True)

print('✓ 环境和组件导入完成')

## 主要代码

完整的代码实现请参考对应的 Python 文件。